In [ ]:
import os
import math
import copy

import numpy as np
import pandas as pd
import platereadertools as prt
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from matplotlib import rc

rc('text',usetex=False)

In [ ]:
# Import Data from the plate readeer experiment we ran on July 7th 2023. This data represents 1 replicate of 80 out of 96
# members of the library as well as MGz1 pATdCRxL1C, MGz1 pATdCRxL1C pEYF1K, and MGz1 Wild Type. The plate map can be found 
# in the p2x11 benchling notebook (https://benchling.com/aztaylor/f/lib_zcNJTcZv-microbiome-persistence-control-p2/etr_dWwaZ9zW-p2x11-gfp-guide-rna-library-/edit)
# specifically the plate layout is Well11.

data0mM, time0mM = prt.Organize('Data/Plate Reader Experiments/p2x11_80memberlibrary0mMIPTG20230707.txt', 8, 12, 18, 3/60,)
data1mM, time1mM = prt.Organize('Data/Plate Reader Experiments/p2x11_80memberlibrary_1mMIPTG.txt', 8, 12, 18, 3/60)
data10mM, time10mM = prt.Organize('Data/Plate Reader Experiments/p2x11_80memberlibrary_10mMIPTG.txt', 8, 12, 18, 3/60)

In [ ]:
reads = list(data0mM.keys())
print(reads)

In [ ]:
#need to swap rows F and G due to a mistake in making the plate.
for read in reads:
    data0mM[read][(5,6),:,:] = data0mM[read][(6,5),:,:]
    data1mM[read][(5,6),:,:] = data1mM[read][(6,5),:,:]
    data10mM[read][(5,6),:,:] = data10mM[read][(6,5),:,:]

In [ ]:
# Blank by subtracting the blank wells H1-H2 and H5-H12. Also adds the associated error calculated by tge standard deviation.
data_blanked_l = [{}, {}, {}]
for read in reads:  
    for i, data in enumerate([data0mM, data1mM, data10mM]):
        data_blanked_l[i][read] = data[read] - np.ones(data[read].shape)*np.mean(np.concatenate((data[read][7,0:2,:],data[read][7,4:,:])))
        data_blanked_l[i][read+'_err'] = np.ones(data[read].shape)*np.std(np.concatenate((data[read][7,0:2,:],data[read][7,4:,:])))
                    


In [ ]:
for data in data_blanked_l:
    for read in reads:
        prt.well_curves(data, time0mM, read)

In [ ]:
data_blanked_l[2].keys()

In [ ]:
for data in data_blanked_l:
    for read in reads:
        data[read][data[read]<0]=0

In [ ]:
# Blank by subtracting the blank wells H1-H2 and H5-H12. Also adds the associated error calculated by tge standard deviation.
data_normed_l = [{}, {}, {}]  
for i, data in enumerate(data_blanked_l):
    data_normed_l[i]['GFP_Normalized'] = data[reads[1]]/(0.01+data[reads[0]])
    data_normed_l[i]['GFP_Normalized_err'] = np.ones(data[read].shape)*np.std(np.concatenate((data[read][7,0:2,:],data[read][7,4:,:])))

In [ ]:
data_normed_l[2].keys()

In [ ]:
times = [{'GFP_Normalized': np.mean((time0mM[reads[0]], time0mM[reads[1]]), axis=0)}, 
         {'GFP_Normalized': np.mean((time1mM[reads[0]], time1mM[reads[1]]), axis=0)}, 
         {'GFP_Normalized': np.mean((time10mM[reads[0]], time10mM[reads[1]]), axis=0)}]
for i, data in enumerate(data_normed_l):
    prt.well_curves(data, times[i], 'GFP_Normalized')

In [ ]:
titles = [
    ['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11'],
    ['s13', 's14', 's15', 's16', 's19', 's20', 's21', 's22', 's25', 's26', 's27', 's28'],
    ['s30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's41', 's42'],
    ['s43', 's44', 's45', 's46', 's47', 's49', 's50', 's51', 's52', 's53', 's54', 's55'],
    ['s56', 's57', 's58', 's59', 's60', 's61', 's62', 's63', 's65', 's66', 's67', 's68'],
    ['s69', 's70', 's71', 's84', 's85', 's86', 's87', 's88', 's90', 's93', 's94', 's95'],
    ['s72', 's74', 's76', 's77', 's78', 's79', 's80', 's81', 's82', 's83', 'dCasRx\npEYF1k', 'Blank'],
    ['Blank', 'Blank', 'pEYF1K', 'Wild Type', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank']
]


In [ ]:
fig, axs = plt.subplots(8,12, sharey=True, sharex=True, figsize=(30,20), layout='tight')
face_size = 0.5
title_fsize = 48
subtitle_fsize = 26
label_fsize = 36
axis_fsize = 18
axs2_l = []

Test = False

for i in range(8):
    for j in range(12):
        axs2 = axs[i,j].twinx()
        if not Test:
            axs2.set_ylim((0,1.5))
            axs2.yaxis.set_tick_params(labelsize=axis_fsize) 
            p21 = axs2.scatter(times[0]['GFP_Normalized'], data_blanked_l[0]['600'][i,j,:], s=face_size, color='silver')
            p22 = axs2.scatter(times[1]['GFP_Normalized'], data_blanked_l[1]['600'][i,j,:], s=face_size, color='gray')
            p23 = axs2.scatter(times[2]['GFP_Normalized'], data_blanked_l[2]['600'][i,j,:], s=face_size, color='Black')
            if j != 11:
                axs2.get_yaxis().set_ticks([])
            p1 = axs[i,j].scatter(times[0]['GFP_Normalized'], data_normed_l[0]['GFP_Normalized'][i,j,:], s=face_size, label='0mM', color='springgreen')
            p2 = axs[i,j].scatter(times[1]['GFP_Normalized'], data_normed_l[1]['GFP_Normalized'][i,j,:], s=face_size, label='1mM', color='forestgreen')
            p3 = axs[i,j].scatter(times[2]['GFP_Normalized'], data_normed_l[2]['GFP_Normalized'][i,j,:], s=face_size, label='10mM', color='darkgreen')
        
        axs[i,j].set_title(titles[i][j], size=subtitle_fsize)
        
        axs[i,j].spines['right'].set_visible(False)
        axs[i,j].spines['top'].set_visible(False)
         
        axs[i,j].yaxis.set_tick_params(labelsize=axis_fsize)
        axs[i,j].xaxis.set_tick_params(labelsize=axis_fsize)  
        
        axs3 = copy.copy(axs2)

plt.subplots_adjust(hspace=0.7)
if Test:
    p1 = mpatches.Patch(color='red', label='The red data')
    p2 = mpatches.Patch(color='red', label='The red data')
    p3 = mpatches.Patch(color='red', label='The red data')
    p21 = mpatches.Patch(color='red', label='The red data')
    p22 = mpatches.Patch(color='red', label='The red data')
    p23 = mpatches.Patch(color='red', label='The red data')        
fig.suptitle('GFP dCasRx Knockdown Library', size=title_fsize)
fig.supxlabel('Time [Hr.]', size=label_fsize, y = 0.05)
fig.supylabel('RFU/(OD$_{600i}$+OD$_{600}}$)', size=label_fsize, x=0.075)
secondylabel = plt.annotate('OD$_{600}$', xy=(0.96, 0.5), xycoords=fig, size=label_fsize, ha='right', va='center', rotation=90)
fig.legend([p1,p2,p3], ['0mM', '1mM', '10mM'], loc='outside upper left', bbox_to_anchor=(0.125,0.95), title='IPTG Conc.',fontsize=axis_fsize, title_fontsize=axis_fsize, markerscale=20, ncol=3)
fig.legend([p21,p22,p23], ['0mM', '1mM', '10mM'], loc='outside upper right', bbox_to_anchor=(0.90,0.95), title='IPTG Conc.',fontsize=axis_fsize, title_fontsize=axis_fsize, markerscale=20, ncol=3)
plt.show()

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,5), layout='tight', sharey=True)
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Spectral(np.linspace(0,1,96)))


face_size = 5.0
for i in range(8):
    for j in range(12):
            scatter1 = axs[0].scatter(times[0]['GFP_Normalized'], data_normed_l[0]['GFP_Normalized'][i,j,:], s=face_size)
            scatter2 = axs[1].scatter(times[1]['GFP_Normalized'], data_normed_l[1]['GFP_Normalized'][i,j,:], s=face_size)
            scatter3 = axs[2].scatter(times[2]['GFP_Normalized'], data_normed_l[2]['GFP_Normalized'][i,j,:], s=face_size)

axs[0].set_title('0mM IPTG')
axs[1].set_title('1mM IPTG')
axs[2].set_title('10mM IPTG')

axs[0].set_xlabel('Time [Hr.]')
axs[1].set_xlabel('Time [Hr.]')
axs[2].set_xlabel('Time [Hr.]')

axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)

axs[0].set_ylabel('$RFU/(OD_{600i}+OD_{600})$)')
ticks = list(np.arange(0,96,1))
plt.colorbar(scatter3, ticks=ticks)

In [ ]:
1/3*20/96

In [ ]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.RdYlGn(np.linspace(0,1,96)))

fig, axs = plt.subplots(1,2, figsize=(20,10), sharey=True)

scale=10
fontsize = 18
tickFontSize = 18
face_size = 10
alpha = 0.75
linewidth = 2.5
rect_l = []
counter = 0
counter2 = 0
counter3 = 0
filter_mode = 'nearest'#'constant'#'mirror'

spacers = []
for row in titles:
    for title in row:
        if title[0] == 's': 
            spacers.append("Spacer " + title.strip('s'))
        elif title == 'dCasRx\npEYF1k' or title == 'pEYF1K' or title == 'Wild Type':
            spacers.append(title)
print(len(spacers))
spacer_l = []
for i in range(8):
    for j in range(12):
        counter3 += 1
        if counter3 < 88 and counter3 != 85 and counter3 != 84 and counter3 !=83:
            x1 = times[0]['GFP_Normalized']
            x2 = times[0]['GFP_Normalized']
            y1 = sp.signal.savgol_filter((data_normed_l[1]['GFP_Normalized'][i,j,:]-data_normed_l[0]['GFP_Normalized'][i,j,:])/data_normed_l[1]['GFP_Normalized'][i,j,:], 15, 5, mode=filter_mode)
            y2 = sp.signal.savgol_filter((data_normed_l[2]['GFP_Normalized'][i,j,:]-data_normed_l[0]['GFP_Normalized'][i,j,:])/data_normed_l[2]['GFP_Normalized'][i,j,:], 15, 5, mode=filter_mode)
            
            scatter1, = axs[0].plot(x1, y1, alpha=alpha, linewidth=linewidth)
            scatter2, = axs[1].plot(x2, y2, alpha=alpha, linewidth=linewidth)

            color = scatter1.get_color()
            
        if counter != 6:
            height = counter*0.2
            width = counter2*0.0055
            counter += 1
            counter2 += 1
        else:
            counter = 0
            height = counter*0.2
            counter += 1
        if counter3 < 82:# and counter3 != 85 and counter3 != 84 and counter3 !=83:
            rect = plt.Rectangle((0.136+(counter3)*0.0044+width,0.85+height*0.2), 0.0328, 0.03, facecolor=color, alpha= 0.5, edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure)
            text = plt.text(0.139+(counter3)*0.0044+width, 0.86+height*0.2, spacers[counter3], fontsize=7.5, clip_on=False, transform=fig.transFigure)
            rect_l.append(rect)
            spacer_l.append(text)
print(len(rect_l))
fig.patches.extend(rect_l+spacer_l)
cds_height = 1.114
cds_xbound = 0.191
rbs = plt.Rectangle((0.14,1.1), 0.05, 0.03, facecolor="gray", edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure)
cds = mpl.patches.FancyArrowPatch((cds_xbound, cds_height), (cds_xbound+0.71, cds_height), arrowstyle='Simple, head_width=0.2, head_length=0.1', facecolor="green", edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure, mutation_scale=110)

rbs_text = plt.text(0.15, 1.105, 'BCD5', fontsize=15, clip_on=False, transform=fig.transFigure)
cds_text = plt.text(0.5, 1.105, 'GFP', fontsize=15, clip_on=False, transform=fig.transFigure)   
fig.patches.extend([rbs]+[cds]+[rbs_text]+[cds_text])

ticks = list(np.arange(0,85,1))
norm = plt.Normalize(0, 85)
sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn, norm=norm)

cbar = fig.colorbar(sm, ax=axs[:], ticks=ticks, orientation='horizontal', location="top", anchor=(0.5, 0.3), shrink=0.96, aspect=40)
cbar.set_ticklabels([])
cbar_bbox = cbar.ax.get_position()
print(cbar_bbox.width)
#cbar.ax.tick_params(length=1000)

axs[0].vlines(5, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')
axs[0].vlines(15, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')

axs[1].vlines(5, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')
axs[1].vlines(15, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')

axs[0].set_title('1mM IPTG Fold Change', fontsize=fontsize)
axs[1].set_title('10mM IPTG Fold Change', fontsize=fontsize)

axs[0].set_xlabel('$Time$ $[Hr.]$', fontsize=fontsize)
axs[1].set_xlabel('$Time$ $[Hr.]$', fontsize=fontsize)

axs[0].tick_params(axis='both', which='major', labelsize=tickFontSize)
axs[1].tick_params(axis='both', which='major', labelsize=tickFontSize)

axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

axs[0].set_ylim(-3,1)


axs[0].set_ylabel('${RFU}/(OD_{600i}+OD_{600})$)', fontsize=fontsize)
plt.savefig("/Users/alec/Documents/GradSchool/Yeung_Lab/gRNA2Knockdown/dCasRx Paper/Figures/80MemberLibraryFoldChangeWithtilling.png", dpi=600)


In [ ]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.RdYlGn(np.linspace(0,1,96)))

fig, axs = plt.subplots(1,2, figsize=(20,10), sharey=True)

scale=10
fontsize = 18
tickFontSize = 18
face_size = 10
alpha = 0.75
linewidth = 2.5
rect_l = []
counter = 0
counter2 = 0
counter3 = 0
filter_mode = 'nearest'#'constant'#'mirror'

spacers = []
for row in titles:
    for title in row:
        if title[0] == 's': 
            spacers.append("Spacer " + title.strip('s'))
        elif title == 'dCasRx\npEYF1k' or title == 'pEYF1K' or title == 'Wild Type':
            spacers.append(title)
print(len(spacers))
spacer_l = []
for i in range(8):
    for j in range(12):
        counter3 += 1
        if counter3 < 88 and counter3 != 85 and counter3 != 84 and counter3 !=83:
            x1 = times[0]['GFP_Normalized']
            x2 = times[0]['GFP_Normalized']
            y1 = (data_normed_l[1]['GFP_Normalized'][i,j,:]-data_normed_l[0]['GFP_Normalized'][i,j,:])/data_normed_l[1]['GFP_Normalized'][i,j,:]
            y2 = (data_normed_l[2]['GFP_Normalized'][i,j,:]-data_normed_l[0]['GFP_Normalized'][i,j,:])/data_normed_l[2]['GFP_Normalized'][i,j,:]
            
            scatter1, = axs[0].plot(x1, y1, alpha=alpha, linewidth=linewidth)
            scatter2, = axs[1].plot(x2, y2, alpha=alpha, linewidth=linewidth)

            color = scatter1.get_color()
            
        if counter != 6:
            height = counter*0.2
            width = counter2*0.0055
            counter += 1
            counter2 += 1
        else:
            counter = 0
            height = counter*0.2
            counter += 1
        if counter3 < 82:# and counter3 != 85 and counter3 != 84 and counter3 !=83:
            rect = plt.Rectangle((0.136+(counter3)*0.0044+width,0.85+height*0.2), 0.0328, 0.03, facecolor=color, alpha= 0.5, edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure)
            text = plt.text(0.139+(counter3)*0.0044+width, 0.86+height*0.2, spacers[counter3], fontsize=7.5, clip_on=False, transform=fig.transFigure)
            rect_l.append(rect)
            spacer_l.append(text)
print(len(rect_l))
fig.patches.extend(rect_l+spacer_l)
cds_height = 1.114
cds_xbound = 0.191
rbs = plt.Rectangle((0.14,1.1), 0.05, 0.03, facecolor="gray", edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure)
cds = mpl.patches.FancyArrowPatch((cds_xbound, cds_height), (cds_xbound+0.71, cds_height), arrowstyle='Simple, head_width=0.2, head_length=0.1', facecolor="green", edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure, mutation_scale=110)

rbs_text = plt.text(0.15, 1.105, 'BCD5', fontsize=15, clip_on=False, transform=fig.transFigure)
cds_text = plt.text(0.5, 1.105, 'GFP', fontsize=15, clip_on=False, transform=fig.transFigure)   
fig.patches.extend([rbs]+[cds]+[rbs_text]+[cds_text])

ticks = list(np.arange(0,85,1))
norm = plt.Normalize(0, 85)
sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn, norm=norm)

cbar = fig.colorbar(sm, ax=axs[:], ticks=ticks, orientation='horizontal', location="top", anchor=(0.5, 0.3), shrink=0.96, aspect=40)
cbar.set_ticklabels([])
cbar_bbox = cbar.ax.get_position()
print(cbar_bbox.width)
#cbar.ax.tick_params(length=1000)

axs[0].vlines(5, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')
axs[0].vlines(15, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')

axs[1].vlines(5, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')
axs[1].vlines(15, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')

axs[0].set_title('1mM IPTG Fold Change', fontsize=fontsize)
axs[1].set_title('10mM IPTG Fold Change', fontsize=fontsize)

axs[0].set_xlabel('$Time$ $[Hr.]$', fontsize=fontsize)
axs[1].set_xlabel('$Time$ $[Hr.]$', fontsize=fontsize)

axs[0].tick_params(axis='both', which='major', labelsize=tickFontSize)
axs[1].tick_params(axis='both', which='major', labelsize=tickFontSize)

axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

axs[0].set_ylim(-3,1)


axs[0].set_ylabel('${RFU}/(OD_{600i}+OD_{600})$)', fontsize=fontsize)
plt.savefig("/Users/alec/Documents/GradSchool/Yeung_Lab/gRNA2Knockdown/dCasRx Paper/Figures/80MemberLibraryFoldChangeWithtilling_non_filtered.png", dpi=600)


In [ ]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.RdYlGn(np.linspace(0,1,96)))

fig, axs = plt.subplots(1,2, figsize=(20,10), sharey=True)

scale=10
fontsize = 18
tickFontSize = 18
face_size = 10
alpha = 0.75
linewidth = 2.5
counter = 0
counter2 = 0
counter3 = 0
timepoint = 7.5 

filter_mode = 'nearest'#'constant'#'mirror'

rect_l = []
spacers = []
conc1FcDp_l = []
conc2FcDp_l = []

for row in titles:
    for title in row:
        if title[0] == 's': 
            spacers.append("Spacer " + title.strip('s'))
        elif title == 'dCasRx\npEYF1k' or title == 'pEYF1K' or title == 'Wild Type':
            spacers.append(title)

spacer_l = []
for i in range(8):
    for j in range(12):
        counter3 += 1
        if counter3 < 88 and counter3 != 85 and counter3 != 84 and counter3 !=83:
            x1 = times[0]['GFP_Normalized']
            x2 = times[0]['GFP_Normalized']
            y1 = sp.signal.savgol_filter((data_normed_l[1]['GFP_Normalized'][i,j,:]-data_normed_l[0]['GFP_Normalized'][i,j,:])/data_normed_l[1]['GFP_Normalized'][i,j,:], 15, 5, mode=filter_mode)
            y2 = sp.signal.savgol_filter((data_normed_l[2]['GFP_Normalized'][i,j,:]-data_normed_l[0]['GFP_Normalized'][i,j,:])/data_normed_l[2]['GFP_Normalized'][i,j,:], 15, 5, mode=filter_mode)
            
            scatter1, = axs[0].plot(x1, y1, alpha=alpha, linewidth=linewidth)
            scatter2, = axs[1].plot(x2, y2, alpha=alpha, linewidth=linewidth)

            color = scatter1.get_color()
            conc1FcDp = y1[int((timepoint*60)/3)] # Fold change datapoint (FcDp)
            conc2FcDp = y2[int((timepoint*60)/3)] # Fold change datapoint (FcDp)

            conc1FcDp_l.append(conc1FcDp)
            conc2FcDp_l.append(conc2FcDp)

cds_height = 1.114
cds_xbound = 0.191
rbs = plt.Rectangle((0.14,1.1), 0.05, 0.03, facecolor="gray", edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure)
cds = mpl.patches.FancyArrowPatch((cds_xbound, cds_height), (cds_xbound+0.71, cds_height), arrowstyle='Simple, head_width=0.2, head_length=0.1', facecolor="green", edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure, mutation_scale=110)

rbs_text = plt.text(0.15, 1.105, 'BCD5', fontsize=15, clip_on=False, transform=fig.transFigure)
cds_text = plt.text(0.5, 1.105, 'GFP', fontsize=15, clip_on=False, transform=fig.transFigure)   
fig.patches.extend([rbs]+[cds]+[rbs_text]+[cds_text])

ticks = list(np.arange(min(conc1FcDp_l), max(conc1FcDp_l)))
norm = plt.Normalize(min(conc1FcDp_l), max(conc1FcDp_l))
sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn, norm=norm)

colors_conc1 = sm.to_rgba(conc1FcDp_l)
colors_conc2 = sm.to_rgba(conc2FcDp_l)

counter = 0
counter2 = 0
counter3 = 0
for i in range(len(conc1FcDp_l)):
    counter3 += 1
    if counter != 6:
        height = counter*0.2
        width = counter2*0.0055
        counter += 1
        counter2 += 1
    else:
        counter = 0
        height = counter*0.2
        counter += 1
    if counter3 < 82:# and counter3 != 85 and counter3 != 84 and counter3 !=83:
        rect = plt.Rectangle((0.136+(counter3)*0.0044+width,0.85+height*0.2), 0.0328, 0.03, facecolor=colors_conc1[i], alpha= 0.5, edgecolor='black', linewidth=1, clip_on=False, transform=fig.transFigure)
        text = plt.text(0.139+(counter3)*0.0044+width, 0.86+height*0.2, spacers[i], fontsize=7.5, clip_on=False, transform=fig.transFigure)
        rect_l.append(rect)
        spacer_l.append(text)
        
fig.patches.extend(rect_l+spacer_l)

cbar = fig.colorbar(sm, ax=axs[:], ticks=ticks, orientation='horizontal', location="top", anchor=(0.5, 0.3), shrink=0.96, aspect=40)
#cbar.set_ticklabels([])
cbar_bbox = cbar.ax.get_position()
#cbar.ax.tick_params(length=1000)

axs[0].vlines(5, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')
axs[0].vlines(15, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')

axs[1].vlines(5, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')
axs[1].vlines(15, axs[0].get_ylim()[0], axs[0].get_ylim()[1], linestyle='--', color='black')

axs[0].set_title('1mM IPTG Fold Change', fontsize=fontsize)
axs[1].set_title('10mM IPTG Fold Change', fontsize=fontsize)

axs[0].set_xlabel('$Time$ $[Hr.]$', fontsize=fontsize)
axs[1].set_xlabel('$Time$ $[Hr.]$', fontsize=fontsize)

axs[0].tick_params(axis='both', which='major', labelsize=tickFontSize)
axs[1].tick_params(axis='both', which='major', labelsize=tickFontSize)

axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

axs[0].set_ylim(-3,1)
axs[0].set_ylabel('${RFU}/(OD_{600i}+OD_{600})$)', fontsize=fontsize)

plt.savefig("/Users/alec/Documents/GradSchool/Yeung_Lab/gRNA2Knockdown/dCasRx Paper/Figures/80MemberLibraryFoldChangeWithtilling2.png", dpi=600)
plt.show()

In [ ]:
# Determine the area under the curve for the fluorescence.
AUC = []
for data in data_normed_l:
    AUC.append(np.trapz(data['GFP_Normalized']))

In [ ]:
fig, axs = plt.subplots(1,3)
for i in range(8):
    for j in range(12):
        ind = j+i*12
        axs[0].bar(ind, AUC[0][i,j])
        axs[1].bar(ind, AUC[1][i,j])
        axs[2].bar(ind, AUC[2][i,j])

In [ ]:

fig, ax = plt.subplots(figsize=(25,10))


margin = 2
 
# set heights of bars
bars1 = AUC[0].reshape((96))
bars2 = AUC[1].reshape((96))
bars3 = AUC[2].reshape((96))

barWidth = (25-2*margin)/len(bars1) 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
# Make the plot
ax.bar(r1, bars1, color='green', width=barWidth, edgecolor='white', label='0mM')
ax.bar(r2, bars2, color='red', width=barWidth, edgecolor='white', label='1mM')
ax.bar(r3, bars3, color='blue', width=barWidth, edgecolor='white', label='10mM')
 
# Add xticks on the middle of the group bars
ax.set_xlabel('Spacer', fontsize=20)
ax.set_ylabel('Total Fluorescence [RFU*Hr.]', fontsize=20)
ax.set_xticks([r + barWidth for r in range(len(bars1))], np.array(titles).reshape(96), rotation= 90, fontsize=10)
 
# Create legend & Show graphic
ax.legend(frameon=False, fontsize=24)
plt.xlim([-1,r1.size])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))

margin = 2
 
# set heights of bars
bars1 = AUC[1].reshape((96))/AUC[0].reshape((96))*100
bars2 = AUC[2].reshape((96))/AUC[0].reshape((96))*100

barWidth = (25-2*margin)/len(bars1) 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]

 
# Make the plot
ax.bar(r1, bars1, color='red', width=barWidth, edgecolor='white', label='1mM')
ax.bar(r2, bars2, color='blue', width=barWidth, edgecolor='white', label='10mM')

 
# Add xticks on the middle of the group bars
ax.set_xlabel('Spacer', fontsize=24)
ax.set_ylabel('Percent of 0mM Normalized Total Fluorescence', fontsize=20)
ax.set_xticks([r + barWidth for r in range(len(bars1))], np.array(titles).reshape(96), rotation=90, fontsize=10)
 
# Create legend & Show graphic
ax.legend(frameon=False, fontsize=24)
plt.xlim([-1,r1.size])
plt.show()

In [ ]:
PerChange_df = pd.DataFrame(columns=['PC1mM', 'PC10mM'])

PerChange_df['PC1mM'] = bars1
PerChange_df['PC10mM'] = bars2
PerChange_df.index = np.array(titles).reshape(96)

In [ ]:
print(PerChange_df[PerChange_df.PC1mM == PerChange_df.PC1mM.min(numeric_only=True)])

In [ ]:
print(PerChange_df[PerChange_df.PC10mM == PerChange_df.PC10mM.min(numeric_only=True)])

In [ ]:
PerChange_df['PC1mM'].nlargest(n=5)

In [ ]:
PerChange_df['PC1mM'].nsmallest(n=5)

In [ ]:
PerChange_df['PC10mM'].nlargest(n=5)

In [ ]:
PerChange_df['PC10mM'].nsmallest(n=5)

In [ ]:
fig, axs = plt.subplots(8,12, sharey=True, sharex=True, figsize=(30,20), layout='tight')
face_size = 0.5
title_fsize = 48
subtitle_fsize = 26
label_fsize = 36
axis_fsize = 18
axs2_l = []

Test = False

for i in range(8):
    for j in range(12):
        axs2 = axs[i,j].twinx()
        if not Test:
            axs2.set_ylim((0,1.5))
            axs2.yaxis.set_tick_params(labelsize=axis_fsize) 
            p21 = axs2.scatter(times[0]['GFP_Normalized'], data_blanked_l[0]['600'][i,j,:], s=face_size, color='silver')
            p22 = axs2.scatter(times[1]['GFP_Normalized'], data_blanked_l[1]['600'][i,j,:], s=face_size, color='gray')
            p23 = axs2.scatter(times[2]['GFP_Normalized'], data_blanked_l[2]['600'][i,j,:], s=face_size, color='Black')
            if j != 11:
                axs2.get_yaxis().set_ticks([])
            p1 = axs[i,j].scatter(times[0]['GFP_Normalized'], data_normed_l[0]['GFP_Normalized'][i,j,:], s=face_size, label='0mM', color='springgreen')
            p2 = axs[i,j].scatter(times[1]['GFP_Normalized'], data_normed_l[1]['GFP_Normalized'][i,j,:], s=face_size, label='1mM', color='forestgreen')
            p3 = axs[i,j].scatter(times[2]['GFP_Normalized'], data_normed_l[2]['GFP_Normalized'][i,j,:], s=face_size, label='10mM', color='darkgreen')

        axs[i,j].set_title(titles[i][j], size=subtitle_fsize)
        axs[i,j].spines['right'].set_visible(False)
        axs[i,j].spines['top'].set_visible(False)
         
        axs[i,j].yaxis.set_tick_params(labelsize=axis_fsize)
        axs[i,j].xaxis.set_tick_params(labelsize=axis_fsize)  
        
        axs3 = copy.copy(axs2)
        
High1mM = ['s93', 's84', 'H7', 's67', 's42']
High10mM = ['s42', 'H7', 's67', 's28', 's39']
Low1mM = ['s3', 's7', 's5', 's62', 's0']
Low10mM = ['s7', 's5', 's61', 's22a', 's68']
title_arr = np.array(titles)

# Find the indices of the elements in the array
indH1mM = [(np.where(title_arr == element)[0][0],np.where(title_arr == element)[1][0])  for element in High1mM]
indH10mM = [(np.where(title_arr == element)[0][0],np.where(title_arr == element)[1][0])  for element in High10mM]
indL1mM = [(np.where(title_arr == element)[0][0],np.where(title_arr == element)[1][0])  for element in Low1mM]
indL10mM = [(np.where(title_arr == element)[0][0],np.where(title_arr == element)[1][0])  for element in Low10mM]

high_colors = ['#DC1C13', '#EA4C46', '#F07470', '#F1959B', '#F6BDC0']
low_colors = ['#0000FF', '#4949FF', '#7879FF', '#A3A3FF', '#BFBFFF']

for i in range(5):
    axs[indH1mM[i][0], indH1mM[i][1]].set_facecolor(high_colors[i])
    axs[indL1mM[i][0], indL1mM[i][1]].set_facecolor(low_colors[i])
    
a=1
for i in range(5):
    axH = axs[indH10mM[i][0], indH10mM[i][1]].axis()
    axL = axs[indL10mM[i][0], indL10mM[i][1]].axis()
    recH = plt.Rectangle((axH[0]-a, axH[2]-a), (axH[1]-axH[0])+2*a, (axH[3]-axH[2])+2*a, edgecolor=high_colors[i], fill=False, lw=10, linestyle='-')
    recL = plt.Rectangle((axL[0]-a, axL[2]-a), (axL[1]-axL[0])+2*a, (axL[3]-axL[2])+2*a, edgecolor=low_colors[i], fill=False, lw=10, linestyle='-')
    recH = axs[indH10mM[i][0], indH10mM[i][1]].add_patch(recH)
    recL = axs[indL10mM[i][0], indL10mM[i][1]].add_patch(recL)
    recH.set_clip_on(False)
    recL.set_clip_on(False)
    
plt.subplots_adjust(hspace=0.7)

if Test:
    p1 = mpatches.Patch(color='red', label='The red data')
    p2 = mpatches.Patch(color='red', label='The red data')
    p3 = mpatches.Patch(color='red', label='The red data')
    p21 = mpatches.Patch(color='red', label='The red data')
    p22 = mpatches.Patch(color='red', label='The red data')
    p23 = mpatches.Patch(color='red', label='The red data') 
           
fig.suptitle('GFP dCasRx Knockdown Library', size=title_fsize)
fig.supxlabel('Time [Hr.]', size=label_fsize, y = 0.05)
fig.supylabel('RFU/(OD$_{600i}$+OD$_{600}}$)', size=label_fsize, x=0.075)
secondylabel = plt.annotate('OD$_{600}$', xy=(0.96, 0.5), xycoords=fig, size=label_fsize, ha='right', va='center', rotation=90)
fig.legend([p1,p2,p3], ['0mM', '1mM', '10mM'], loc='outside upper left', bbox_to_anchor=(0.125,0.95), title='IPTG Conc.',fontsize=axis_fsize, title_fontsize=axis_fsize, markerscale=20, ncol=3)
fig.legend([p21,p22,p23], ['0mM', '1mM', '10mM'], loc='outside upper right', bbox_to_anchor=(0.90,0.95), title='IPTG Conc.',fontsize=axis_fsize, title_fontsize=axis_fsize, markerscale=20, ncol=3)
plt.show()

In [ ]:
# I lets see how the distribution of AUC changes.
flatAUC = [data.flatten() for data in AUC]
conc = ['0mM', '1mM', '10mM']

fig, ax = plt.subplots(figsize=(9,6))
for data in flatAUC:
    ax.hist(data[:81], bins=150, histtype='barstacked')
ax.legend(conc, frameon=False)

ax.set_xlabel('Total Florescence [RFU*Hr]', fontsize=12)
ax.set_ylabel('Count',fontsize=12)

In [ ]:
# Lets look at the distribution change of the percentage of 0mM Flourescence
flatPercent = [data.flatten() for data in [bars1, bars2]]
conc = ['1mM', '10mM']
fig, ax = plt.subplots(figsize=(9,6))
for data in flatPercent:
    ax.hist(data[:81], bins=150, histtype='barstacked')
ax.legend(conc, frameon=False)

ax.set_xlabel('Fluorescence Percentage of 0mM Case', fontsize=12)
ax.set_ylabel('Count', fontsize=12)